In [31]:
import os
os.chdir('D:\8th SEMESTER\ML LAB\ML Lab Spring 25')

<>:2: SyntaxWarning: invalid escape sequence '\8'
<>:2: SyntaxWarning: invalid escape sequence '\8'
C:\Users\Hassan Laptop Point\AppData\Local\Temp\ipykernel_14184\1283731416.py:2: SyntaxWarning: invalid escape sequence '\8'
  os.chdir('D:\8th SEMESTER\ML LAB\ML Lab Spring 25')


In [32]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, explained_variance_score, r2_score
from timeseires.utils.to_split import to_split
from timeseires.utils.multivariate_multi_step import multivariate_multi_step
from timeseires.utils.multivariate_single_step import multivariate_single_step
from timeseires.utils.univariate_multi_step import univariate_multi_step
from timeseires.utils.univariate_single_step import univariate_single_step
from timeseires.utils.CosineAnnealingLRS import CosineAnnealingLRS
from timeseires.callbacks.EpochCheckpoint import EpochCheckpoint
from tensorflow.keras.callbacks import ModelCheckpoint
from timeseires.callbacks.TrainingMonitor import TrainingMonitor
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import LSTM, Bidirectional, Add
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv1D,TimeDistributed
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten,MaxPooling1D,Concatenate,AveragePooling1D, GlobalMaxPooling1D, Input
from tensorflow.keras.models import Sequential,Model
import pandas as pd
import time, pickle
import numpy as np
import tensorflow.keras.backend as K
import tensorflow
from tensorflow.keras.layers import Input, Reshape, Lambda
from tensorflow.keras.layers import Layer, Flatten, LeakyReLU, concatenate, Dense
from tensorflow.keras.regularizers import l2
import glob
import h5py
import matplotlib.pyplot as plt
from keras.callbacks import Callback

In [33]:
#lookback = 24
model = None
start_epoch = 0
time_steps=24
num_features=21

In [34]:
def create_lstm():
    input_data = Input(shape=(time_steps, num_features))
    lstm_layer1 = LSTM(8, return_sequences=True)(input_data)
    lstm_layer2 = LSTM(20)(lstm_layer1)
    x = Flatten()(lstm_layer2)
    output_data = Dense(1)(x)
    model = Model(input_data, output_data)
    return model

In [35]:
model1 = create_lstm()
model1.summary()

Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_4 (InputLayer)      │ (None, 24, 21)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_8 (LSTM)                   │ (None, 24, 8)          │           960 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_9 (LSTM)                   │ (None, 20)             │         2,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_4 (Flatten)             │ (None, 20)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 1)              │            21 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,301 (12.89 KB)

 Trainable params: 3,301 (12.89 KB)

 Non-trainable params: 0 (0.00 B)

In [36]:
tensorflow.keras.utils.plot_model(model1 )

You must install pydot (`pip install pydot`) for `plot_model` to work.


In [37]:
checkpoints = r'D:\8th SEMESTER\ML LAB\ML Lab Spring 25\LAB 10\lab10\\E1-cp-{epoch:04d}-loss{val_loss:.2f}.h5'
OUTPUT_PATH = r'D:\8th SEMESTER\ML LAB\ML Lab Spring 25\LAB 10'
FIG_PATH = os.path.sep.join([OUTPUT_PATH,"\history.png"])
JSON_PATH = os.path.sep.join([OUTPUT_PATH,"\history.json"])

<>:3: SyntaxWarning: invalid escape sequence '\h'
<>:4: SyntaxWarning: invalid escape sequence '\h'
<>:3: SyntaxWarning: invalid escape sequence '\h'
<>:4: SyntaxWarning: invalid escape sequence '\h'
C:\Users\Hassan Laptop Point\AppData\Local\Temp\ipykernel_14184\3975183762.py:3: SyntaxWarning: invalid escape sequence '\h'
  FIG_PATH = os.path.sep.join([OUTPUT_PATH,"\history.png"])
C:\Users\Hassan Laptop Point\AppData\Local\Temp\ipykernel_14184\3975183762.py:4: SyntaxWarning: invalid escape sequence '\h'
  JSON_PATH = os.path.sep.join([OUTPUT_PATH,"\history.json"])


In [38]:
# construct the callback to save only the *best* model to disk
# based on the validation loss
EpochCheckpoint1 = ModelCheckpoint(checkpoints,
                             monitor="val_loss",
                             save_best_only=True, 
                             verbose=1)
TrainingMonitor1=TrainingMonitor(FIG_PATH, jsonPath=JSON_PATH, startAt=start_epoch)

# construct the set of callbacks
callbacks = [EpochCheckpoint1,TrainingMonitor1]

In [39]:
# if there is no specific model checkpoint supplied, then initialize
# the network and compile the model
if model is None:
    print("[INFO] compiling model...")
    model =create_lstm()
    opt = Adam(1e-3)
    model.compile(loss= 'mae', optimizer=opt, metrics=["mae", "mape"])
# otherwise, load the checkpoint from disk
else:
    print("[INFO] loading {}...".format(model))
    model = load_model(model)

    # update the learning rate
    print("[INFO] old learning rate: {}".format(K.get_value(model.optimizer.lr)))
    K.set_value(model.optimizer.lr, 1e-4)
    print("[INFO] new learning rate: {}".format(K.get_value(model.optimizer.lr)))

[INFO] compiling model...


In [40]:
import os
path_dataset ='D:\\8th SEMESTER\\ML LAB\\ML Lab Spring 25\\AEP_hourly\\processed'
path_tr = os.path.join(path_dataset, 'D:\8th SEMESTER\ML LAB\ML Lab Spring 25\AEP_train.csv')
df_tr = pd.read_csv(path_tr)
train_set = df_tr.iloc[:].values
path_v = os.path.join(path_dataset, 'D:\8th SEMESTER\ML LAB\ML Lab Spring 25\AEP_validation.csv')
df_v = pd.read_csv(path_v)
validation_set = df_v.iloc[:].values 
path_te = os.path.join(path_dataset, 'D:\8th SEMESTER\ML LAB\ML Lab Spring 25\AEP_test.csv')
df_te = pd.read_csv(path_te)
test_set = df_te.iloc[:].values 

path_scaler = os.path.join(path_dataset, 'D:\8th SEMESTER\ML LAB\ML Lab Spring 25\AEP_Scaler.pkl')
scaler         = pickle.load(open(path_scaler, 'rb'))

train_set.shape, validation_set.shape, test_set.shape

<>:3: SyntaxWarning: invalid escape sequence '\8'
<>:6: SyntaxWarning: invalid escape sequence '\8'
<>:9: SyntaxWarning: invalid escape sequence '\8'
<>:13: SyntaxWarning: invalid escape sequence '\8'
<>:3: SyntaxWarning: invalid escape sequence '\8'
<>:6: SyntaxWarning: invalid escape sequence '\8'
<>:9: SyntaxWarning: invalid escape sequence '\8'
<>:13: SyntaxWarning: invalid escape sequence '\8'
C:\Users\Hassan Laptop Point\AppData\Local\Temp\ipykernel_14184\2891296447.py:3: SyntaxWarning: invalid escape sequence '\8'
  path_tr = os.path.join(path_dataset, 'D:\8th SEMESTER\ML LAB\ML Lab Spring 25\AEP_train.csv')
C:\Users\Hassan Laptop Point\AppData\Local\Temp\ipykernel_14184\2891296447.py:6: SyntaxWarning: invalid escape sequence '\8'
  path_v = os.path.join(path_dataset, 'D:\8th SEMESTER\ML LAB\ML Lab Spring 25\AEP_validation.csv')
C:\Users\Hassan Laptop Point\AppData\Local\Temp\ipykernel_14184\2891296447.py:9: SyntaxWarning: invalid escape sequence '\8'
  path_te = os.path.join(pa

((84907, 21), (24259, 21), (12130, 21))

In [41]:
time_steps=24
num_features=21

In [42]:
start = time.time()
train_X , train_y = univariate_multi_step(train_set, time_steps, target_col=0,target_len=1)
validation_X, validation_y = univariate_multi_step(validation_set, time_steps, target_col=0,target_len=1)
test_X, test_y = univariate_multi_step(test_set, time_steps, target_col=0,target_len=1)
print('Time Consumed', time.time()-start, "sec")

Time Consumed 0.9107537269592285 sec


In [43]:
epochs = 10
verbose = 1 #0
batch_size = 32
History = model.fit(train_X,
                        train_y,
                        batch_size=batch_size,   
                        epochs = epochs, 
                        validation_data = (validation_X,validation_y),
                        callbacks=callbacks,verbose = verbose)

Epoch 1/10
2651/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: 0.0646 - mae: 0.0646 - mape: 2286.0879
Epoch 1: val_loss improved from inf to 0.02399, saving model to D:\8th SEMESTER\ML LAB\ML Lab Spring 25\LAB 10\lab10\\E1-cp-0001-loss0.02.h5


2653/2653 ━━━━━━━━━━━━━━━━━━━━ 80s 26ms/step - loss: 0.0646 - mae: 0.0646 - mape: 2285.0854 - val_loss: 0.0240 - val_mae: 0.0240 - val_mape: 10.2906
Epoch 2/10
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: 0.0213 - mae: 0.0213 - mape: 12.3269
Epoch 2: val_loss improved from 0.02399 to 0.01417, saving model to D:\8th SEMESTER\ML LAB\ML Lab Spring 25\LAB 10\lab10\\E1-cp-0002-loss0.01.h5


2653/2653 ━━━━━━━━━━━━━━━━━━━━ 58s 22ms/step - loss: 0.0213 - mae: 0.0213 - mape: 12.4384 - val_loss: 0.0142 - val_mae: 0.0142 - val_mape: 6.1389
Epoch 3/10
2652/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: 0.0134 - mae: 0.0134 - mape: 474.2325
Epoch 3: val_loss improved from 0.01417 to 0.01065, saving model to D:\8th SEMESTER\ML LAB\ML Lab Spring 25\LAB 10\lab10\\E1-cp-0003-loss0.01.h5


2653/2653 ━━━━━━━━━━━━━━━━━━━━ 59s 22ms/step - loss: 0.0134 - mae: 0.0134 - mape: 474.2133 - val_loss: 0.0107 - val_mae: 0.0107 - val_mape: 5.1169
Epoch 4/10
2651/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: 0.0110 - mae: 0.0110 - mape: 85.1081
Epoch 4: val_loss improved from 0.01065 to 0.00871, saving model to D:\8th SEMESTER\ML LAB\ML Lab Spring 25\LAB 10\lab10\\E1-cp-0004-loss0.01.h5


2653/2653 ━━━━━━━━━━━━━━━━━━━━ 61s 23ms/step - loss: 0.0110 - mae: 0.0110 - mape: 85.1164 - val_loss: 0.0087 - val_mae: 0.0087 - val_mape: 3.7373
Epoch 5/10
2651/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: 0.0096 - mae: 0.0096 - mape: 176.9598
Epoch 5: val_loss did not improve from 0.00871
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 59s 22ms/step - loss: 0.0096 - mae: 0.0096 - mape: 176.9739 - val_loss: 0.0091 - val_mae: 0.0091 - val_mape: 3.8302
Epoch 6/10
2652/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: 0.0092 - mae: 0.0092 - mape: 203.1064
Epoch 6: val_loss did not improve from 0.00871
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 58s 22ms/step - loss: 0.0092 - mae: 0.0092 - mape: 203.0428 - val_loss: 0.0094 - val_mae: 0.0094 - val_mape: 4.5280
Epoch 7/10
2651/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: 0.0087 - mae: 0.0087 - mape: 54.8468
Epoch 7: val_loss improved from 0.00871 to 0.00829, saving model to D:\8th SEMESTER\ML LAB\ML Lab Spring 25\LAB 10\lab10\\E1-cp-0007-loss0.01.h5


2653/2653 ━━━━━━━━━━━━━━━━━━━━ 89s 24ms/step - loss: 0.0087 - mae: 0.0087 - mape: 54.8610 - val_loss: 0.0083 - val_mae: 0.0083 - val_mape: 3.6821
Epoch 8/10
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - loss: 0.0084 - mae: 0.0084 - mape: 19.2189
Epoch 8: val_loss improved from 0.00829 to 0.00822, saving model to D:\8th SEMESTER\ML LAB\ML Lab Spring 25\LAB 10\lab10\\E1-cp-0008-loss0.01.h5


2653/2653 ━━━━━━━━━━━━━━━━━━━━ 80s 30ms/step - loss: 0.0084 - mae: 0.0084 - mape: 19.2672 - val_loss: 0.0082 - val_mae: 0.0082 - val_mape: 3.7137
Epoch 9/10
2652/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 0.0082 - mae: 0.0082 - mape: 40.4729
Epoch 9: val_loss improved from 0.00822 to 0.00815, saving model to D:\8th SEMESTER\ML LAB\ML Lab Spring 25\LAB 10\lab10\\E1-cp-0009-loss0.01.h5


2653/2653 ━━━━━━━━━━━━━━━━━━━━ 63s 24ms/step - loss: 0.0082 - mae: 0.0082 - mape: 40.5704 - val_loss: 0.0081 - val_mae: 0.0081 - val_mape: 3.3063
Epoch 10/10
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: 0.0081 - mae: 0.0081 - mape: 259.0497
Epoch 10: val_loss improved from 0.00815 to 0.00812, saving model to D:\8th SEMESTER\ML LAB\ML Lab Spring 25\LAB 10\lab10\\E1-cp-0010-loss0.01.h5


2653/2653 ━━━━━━━━━━━━━━━━━━━━ 79s 23ms/step - loss: 0.0081 - mae: 0.0081 - mape: 259.0190 - val_loss: 0.0081 - val_mae: 0.0081 - val_mape: 3.6120


In [60]:
from tensorflow.keras.models import load_model

# Correct way to assign and load the model
model_path = r'D:\8th SEMESTER\ML LAB\ML Lab Spring 25\lab10\E1-cp-0003-loss0.01.h5'

# Check if the file exists
if os.path.exists(model_path):
    print("File found. Loading model...")
    model = load_model(model_path)
else:
    print("File not found.")
    model = None  # Optional: safe fallback

# Only call predict if model was successfully loaded
if model:
    y_pred_scaled   = model.predict(test_X)
    y_pred          = scaler.inverse_transform(y_pred_scaled)
    y_test_unscaled = scaler.inverse_transform(test_y)

    # Continue with evaluation...
    MAE = np.mean(abs(y_pred - y_test_unscaled)) 
    print('Mean Absolute Error (MAE):', round(MAE, 2))
    # etc...


File not found.


In [78]:
checkpoints = r'D:\8th SEMESTER\ML LAB\ML Lab Spring 25\LAB 10\checkpoints\E2-cp-{epoch:04d}-loss{val_loss:.2f}.h5'
model=r'D:\8th SEMESTER\ML LAB\ML Lab Spring 25\LAB 10\checkpoints\E1-cp-0003-loss0.01.h5'
start_epoch= 34

In [79]:
# construct the callback to save only the *best* model to disk
# based on the validation loss
EpochCheckpoint1 = ModelCheckpoint(checkpoints,
                             monitor="val_loss",
                             save_best_only=True, 
                             verbose=1)
TrainingMonitor1=TrainingMonitor(FIG_PATH, jsonPath=JSON_PATH, startAt=start_epoch)

# construct the set of callbacks
callbacks = [EpochCheckpoint1,TrainingMonitor1]
# if there is no specific model checkpoint supplied, then initialize
# the network and compile the model
if model is None:
    print("[INFO] compiling model...")
    model = PC.build(time_steps=24, num_features=21, reg=0.0005)
    opt = Adam(1e-3)
    model.compile(loss= 'mae', optimizer=opt, metrics=["mae", "mape"])
# otherwise, load the checkpoint from disk
else:
    print("[INFO] loading {}...".format(model))
    model = load_model(model)

    # update the learning rate
    print("[INFO] old learning rate: {}".format(K.get_value(model.optimizer.lr)))
    K.set_value(model.optimizer.lr, 1e-4)
    print("[INFO] new learning rate: {}".format(K.get_value(model.optimizer.lr)))

[INFO] loading D:\8th SEMESTER\ML LAB\ML Lab Spring 25\LAB 10\checkpoints\E1-cp-0003-loss0.01.h5...


FileNotFoundError: [Errno 2] Unable to open file (unable to open file: name = 'D:\8th SEMESTER\ML LAB\ML Lab Spring 25\LAB 10\checkpoints\E1-cp-0003-loss0.01.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [67]:
epochs = 10
verbose = 1 #0
batch_size = 32
History = model.fit(train_X,
                        train_y,
                        batch_size=batch_size,   
                        epochs = epochs, 
                        validation_data = (validation_X,validation_y),
                        callbacks=callbacks,
                        verbose = verbose)

AttributeError: 'str' object has no attribute 'fit'

In [51]:

model = load_model(r'D:\8th SEMESTER\ML LAB\ML Lab Spring 25\E2-cp-0008-loss0.01.h5')

y_pred_scaled   = model.predict(test_X)
y_pred          = scaler.inverse_transform(y_pred_scaled)
y_test_unscaled = scaler.inverse_transform(test_y)
# Mean Absolute Error (MAE)
MAE = np.mean(abs(y_pred - y_test_unscaled)) 
print('Mean Absolute Error (MAE): ' + str(np.round(MAE, 2)))

# Median Absolute Error (MedAE)
MEDAE = np.median(abs(y_pred - y_test_unscaled))
print('Median Absolute Error (MedAE): ' + str(np.round(MEDAE, 2)))

# Mean Squared Error (MSE)
MSE = np.square(np.subtract(y_pred, y_test_unscaled)).mean()
print('Mean Squared Error (MSE): ' + str(np.round(MSE, 2)))

# Root Mean Squarred Error (RMSE) 
RMSE = np.sqrt(np.mean(np.square(y_pred - y_test_unscaled)))
print('Root Mean Squared Error (RMSE): ' + str(np.round(RMSE, 2)))

# Mean Absolute Percentage Error (MAPE)
MAPE = np.mean((np.abs(np.subtract(y_test_unscaled, y_pred)/ y_test_unscaled))) * 100
print('Mean Absolute Percentage Error (MAPE): ' + str(np.round(MAPE, 2)) + ' %')

# Median Absolute Percentage Error (MDAPE)
MDAPE = np.median((np.abs(np.subtract(y_test_unscaled, y_pred)/ y_test_unscaled))) * 100
print('Median Absolute Percentage Error (MDAPE): ' + str(np.round(MDAPE, 2)) + ' %')

print('\n\ny_test_unscaled.shape= ',y_test_unscaled.shape)
print('y_pred.shape= ',y_pred.shape)

FileNotFoundError: [Errno 2] Unable to open file (unable to open file: name = 'D:\8th SEMESTER\ML LAB\ML Lab Spring 25\E2-cp-0008-loss0.01.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)